## Problem Rosalind.info

http://rosalind.info/problems/glob/

To penalize symbol substitutions differently depending on which two symbols are involved in the substitution, we obtain a scoring matrix S in which Si,j represents the (negative) score assigned to a substitution of the i th symbol of our alphabet A with the j th symbol of A.

A gap penalty is the component deducted from alignment score due to the presence of a gap. A gap penalty may be a function of the length of the gap; for example, a linear gap penalty is a constant g such that each inserted or deleted symbol is charged g; as a result, the cost of a gap of length L is equal to gL
.
Given: Two protein strings s and t.
Return: The maximum alignment score between s and t. Use:The BLOSUM62 scoring matrix.
Linear gap penalty equal to 5 (i.e., a cost of -5 is assessed for each gap symbol).

In [2]:
with open("data/BLOSUM62.txt", "r", encoding="utf8") as f:
  lines = f.readlines()

In [3]:
blosum62 = {}
for item in lines:
  list_item = str(item.replace('\n',"")).strip().split()
  blosum62[(list_item[0], list_item[1])] = int(list_item[2])

In [4]:
def print_matrix(matrix, seq1, seq2):
  # Print matrix
  header = "   |   | " + " | ".join(seq2) + " |"
  separator = "-" * len(header)
  print(header)
  print(separator)
  for i in range(len(seq1)+1):
      if i == 0:
          row = " | "
      else:
          row = seq1[i-1] + " | "
      row += " | ".join(str(x).rjust(2) for x in matrix[i])
      row += " |"
      print(row)
  print(separator)

In [5]:
def get_matrix(seq1, seq2, gap_penalty):

  n = len(seq1)
  m = len(seq2)
  matrix = [[0 for i in range(m+1)] for j in range(n+1)]

  # Fill matrix
  for i in range(1, n+1):
      matrix[i][0] = matrix[i-1][0] - gap_penalty

  for j in range(1, m+1):
      matrix[0][j] = matrix[0][j-1] - gap_penalty


  # Fill in rest of matrix using dynamic programming algorithm
  for i in range(1, n+1):
      for j in range(1, m+1):
          match_ = matrix[i-1][j-1] + blosum62.get((seq1[i-1], seq2[j-1]))
          delete = matrix[i-1][j] - gap_penalty
          insert = matrix[i][j-1] - gap_penalty
          matrix[i][j] = max(match_, delete, insert)

  return matrix

In [6]:
def maximum_alignment_score(seq1, seq2, gap_penalty):
    n = len(seq1)
    m = len(seq2)

    # Initialize matrix
    matrix = get_matrix(seq1, seq2, gap_penalty)


    # Print matrix
    if n<16:
      print_matrix(matrix, seq1, seq2)

    # Return the alignment score
    return matrix[n][m]


In [7]:
def optimal_alignment_score(seq1, seq2, gap_penalty):


    n = len(seq1)
    m = len(seq2)

    # Initialize matrix
    matrix = get_matrix(seq1, seq2, gap_penalty)


    # Print matrix
    if n<16:
      print_matrix(matrix, seq1, seq2)


    # Trace back through matrix to find optimal alignment and score
    aligned_seq1 = ""
    aligned_seq2 = ""
    i = n
    j = m
    score = matrix[n][m]
    while i > 0 or j > 0:
      if i > 0 and j > 0 and matrix[i][j] == matrix[i-1][j-1] + blosum62.get((seq1[i-1], seq2[j-1])):
        aligned_seq1 = seq1[i-1] + aligned_seq1
        aligned_seq2 = seq2[j-1] + aligned_seq2
        i -= 1
        j -= 1
      elif i > 0 and matrix[i][j] == matrix[i-1][j] - gap_penalty:
        aligned_seq1 = seq1[i-1] + aligned_seq1
        aligned_seq2 = "-" + aligned_seq2
        i -= 1
      else:
        aligned_seq1 = "-" + aligned_seq1
        aligned_seq2 = seq2[j-1] + aligned_seq2
        j -= 1

    return score, aligned_seq1, aligned_seq2

In [8]:
# Read input sequences
protein1= """CLRHMKMTGYYIAEVDKIPFYNNLAGVAWTIGGRKFWQRQYLTKGSFAKETYGYFGIFGK
STPFLLWYKWPMGGILPHPYNPRRHAFKDGIYMFHNAPDFIHGTKPKVFFRQVSRDMARQ
IMLSHEDLHMWLMIMQNYVVCITFAEKDWEFSGTCWMKTMFWYLHFHCIRWKNDYTIGWN
QLLRFFDPEINMCVDMLCYSCIQLIPANFYAASMTGKISKGAQFTSCGEDCYCDKFIHMK
IFWGYDPQGDLGMYGVNANEKHGLYLCMPLALVYCQHQRFIGMVSGPMDAPCKYLADKDL
QKYIYDISVKEMSVYLLCLGHIKDSGILYGNRLLKKERVAGTMCTMQFDIFDMSYADCHN
NDKTRNRGLSTLKRSWKQYVYIEFGTVPRWVQHATWDLYTNRFFAIMDEFEFLKLRAEKH
WFVDRMWPCMLNIMWSMQMCCVIQFRRCIEMMQCYAFGFEIVWATMRFKRNQAAMWHMYR
RYIPCMDYWDCNGANAAEFCDERESLFKREMLFARAPSCDIEPPTLAQWMFRAVMMLKIF
FWQDIHYACTPDHDEYLWINKSWRMTSDVCFVHWDFWNTGCIYAAPCENKTVPWAKYLYA
SASYNWEDSCMCWLQWLYTRFRNFENEDLPETPACAKAMNYNWALTQVVYYCKILKRQNT
MLGAEVCKTLYTDADPHGLNDLQMQAHCDKWSFNSIAEDRCCPNHLFAQHYDCCQVSRGI
FREYSEHWGFCKHGEKCLRLTQLTTCQPTTAGRGTEKVLTQSWWVCNNFWELCALSACHL
MQYPYECYTKRRHWAEAISLCGQYKSMTQYWLRYNLNVQQQLPAWGKWAAIEIVEFNMCP
WHHKFCVFPPRWYWNHKKWYVFIERWWHSNNLP"""

protein2 = """CLRLMKMTGYYIAEVPFYNNDRWLHCASQAWTIGGRKFWERQDHNEMRCDETGIFGKSTP
FLLWMSGLDMFGWCEKPKTIHVFKDGIYMFHNAGTKKFCHWIKQTAAWWEHFRQVSRDMA
RQNMHYFFNALTLSWTCCITFAFLEKMWEFSGMCWMKTFWYLHFHCIGTFNINQWLRFFD
PETNMHVDMLCWAASSIALIPARFTCCMAASMTGNLQEISGGAEFTSMGAAIKFADCYCD
KFWGYDPQGKMGCMSHWLCMYGVNSNEMPLALVYCRVHNHQRKFISHHDYEPFAPCKIQL
PDKNHFQSIQLQYCRNRQIKKYIYDISVKEMSGYLAAFRDNMCLGHIKMSEMHVQYMLCI
LYLNRLLKKERVEGGMCTMQQEDIFDMSNHHIYKTRMKNKKRSWKQYVYIEFYTVVQRPT
RRWVQHATWDLYENRFFAIMLEEFCWFLFLKLRAEKHWFQDRMWWDWRCMLNICVIQLRL
CIEMMQCEIVWTELFTTMRFHMYRRYIPYWDGYTNGANADEPESLFKREMLFACAPSCDT
EPKYIEIKAQVSESSQFRAVMMGHMNIYAYLKIFSWTFLRRIHYDFKLCTLSINKSWRMT
SDVCFWNTGCIYAAPHLEHHFADLVFWYLYAFASYNWEDSCMCWLGWFENIPYWWDLPMN
YAFKYYCKILKSQNTMLGGNFSDEVCKTLIQKYYVAADPHGLNDIQLAEDTALCGAHCDK
WSFNLIHLFVQHYDCCQVSRGIFRPLQNLLYRLYSEHTGLCKWGEKCLRLTQLTTCQEVH
AAIYTTAGRAMMCFEKMLTQSWWVCNNPWELCALSACHLMQDTRKNYECYTKRRHWAQEY
KSMWQYNLNVQDQLPAIKMGKDEDVLPEIVPFNMRPWHFENACNKYMKFCVFPPRWYWNC
KKKLIERWWHSNNQP"""

protein1 = str(protein1.replace('\n',"")).strip()
protein2 = str(protein2.replace('\n',"")).strip()

In [9]:
# Compute alignment score
score = maximum_alignment_score('PLEASANTLY', "MEANLY", gap_penalty=5)
print("maximum alignment score", score)

   |   | M | E | A | N | L | Y |
--------------------------------
 |  0 | -5 | -10 | -15 | -20 | -25 | -30 |
P | -5 | -2 | -6 | -11 | -16 | -21 | -26 |
L | -10 | -3 | -5 | -7 | -12 | -12 | -17 |
E | -15 | -8 |  2 | -3 | -7 | -12 | -14 |
A | -20 | -13 | -3 |  6 |  1 | -4 | -9 |
S | -25 | -18 | -8 |  1 |  7 |  2 | -3 |
A | -30 | -23 | -13 | -4 |  2 |  6 |  1 |
N | -35 | -28 | -18 | -9 |  2 |  1 |  4 |
T | -40 | -33 | -23 | -14 | -3 |  1 | -1 |
L | -45 | -38 | -28 | -19 | -8 |  1 |  0 |
Y | -50 | -43 | -33 | -24 | -13 | -4 |  8 |
--------------------------------
maximum alignment score 8


In [10]:
# Compute alignment score
score = maximum_alignment_score(protein1, protein2, gap_penalty=5)
print("maximum alignment score", score)

maximum alignment score 1953
